<a href="https://colab.research.google.com/github/hoonerg/AO_DL/blob/main/AO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
#from netCDF4 import Dataset
import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout
import os
from numpy import load

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [ ]:
main_dir ='/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
# input # Train_set
f = load(main_dir+'data/train_set.npy','r') # [600,72,36,4]
tr_x = f
tr_tdim, xdim, ydim, zdim = tr_x.shape
del f

# label [600,1]
f = load(main_dir+'data/train_label.npy','r')
tr_y = f
del f

In [ ]:
# input # Valid_set
f = load(main_dir+'data/valid_set.npy', 'r') # [120,72,36,4]
val_x = f
val_tdim, xdim, ydim, zdim = val_x.shape
del f

# label [120,1]
f = load(main_dir+'data/valid_label.npy', 'r')
val_y = f
del f

In [ ]:
# input # test_set
f = load(main_dir+'data/test_set.npy', 'r') # [120,72,36,4]
test_x = f
test_tdim, xdim, ydim, zdim = test_x.shape
del f

In [ ]:
# set callback
callbacks_list = [

  keras.callbacks.EarlyStopping(
      monitor='val_loss',
      mode='min',
      patience=700,
  ),

  keras.callbacks.ModelCheckpoint(
      filepath=main_dir+'output/cnn/model.hdf5',
      monitor='val_loss',
      save_best_only=True,
  )
]

In [ ]:
# CNN model architecture
inputs = keras.Input(shape=(xdim,ydim,zdim))

conv1 = keras.layers.Conv2D(35, [3,3], activation='tanh', padding='same', strides=1)(inputs)
pool1 = keras.layers.MaxPool2D((2,2), strides=2, padding='same')(conv1)
conv2 = keras.layers.Conv2D(35, [3,3], activation='tanh', padding='same', strides=1)(pool1)
pool2 = keras.layers.MaxPool2D((2,2), strides=2, padding='same')(conv2)
conv3 = keras.layers.Conv2D(35, [3,3], activation='tanh', padding='same', strides=1)(pool2)
flat = keras.layers.Flatten()(conv3)
dense1 = keras.layers.Dense(50, activation='tanh')(flat)

outputs = keras.layers.Dense(1, activation=None)(dense1)

In [ ]:
# Compile
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), 
              loss='mse')

# training
history = model.fit(tr_x, tr_y, batch_size=200, epochs=800, verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(val_x, val_y))

In [ ]:
# model save
model.save(main_dir+'output/cnn/model_last.hdf5')

In [ ]:
# model summary
with open(main_dir+'output/cnn/model_summary.md', 'w') as f:
    with redirect_stdout(f):
        model.summary()

tr_loss = history.history['loss']
tr_loss = np.array(tr_loss)
tr_loss.astype('float32').tofile(main_dir+'output/cnn/tr_loss.gdat')

In [ ]:
fcst = model.predict(test_x)

# save
fcst = np.array(fcst)
tdim = len(fcst)
fcst.astype('float32').tofile(main_dir+'output/cnn/fcst.gdat')

f = open(main_dir+'output/cnn/fcst.ctl','w')
f.write(
'dset ^fcst.gdat\n\
undef -9.99e+08\n\
xdef   1  linear   0.  2.5\n\
ydef   1  linear -90.  2.5\n\
zdef   1  linear 1 1\n\
tdef '+str(tdim)+'  linear jan1980 1yr\n\
vars   1\n\
nino    1   1  Nino3.4\n\
ENDVARS\n'
)
f.close()

os.system('cdo -f nc import_binary '+main_dir+'output/cnn/fcst.ctl '+main_dir+'output/cnn/fcst.nc')
os.system('rm -f '+main_dir+'output/cnn/fcst.ctl '+main_dir+'output/cnn/fcst.gdat')

In [ ]:
print(fcst)